<a href="https://colab.research.google.com/github/debaghtk/finetuning-poc/blob/main/finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://colab.research.google.com/github/huggingface/notebooks/blob/main/peft_docs/en/tensorflow/clm-prompt-tuning.ipynb#scrollTo=buySkzBqU1zS

In [ ]:
!pip install transformers datasets


download model and dataset

In [29]:
from transformers import AutoModelForCausalLM as AutoModel, AutoTokenizer
from datasets import load_dataset

model_name = "bigscience/bloomz-560m"

model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

dataset = load_dataset("ought/raft", "twitter_complaints")

doing inference like TDD

In [44]:
inputs = tokenizer(
    "Tweet text: @nationalgridus I have no water and the bill is current and paid. Can you do something about it? Label:",
    return_tensors="pt",
)

output = model.generate(**inputs)
print(tokenizer.batch_decode(output, skip_special_tokens=True))

['Tweet text: @nationalgridus I have no water and the bill is current and paid. Can you do something about it? Label: National']


preprocessing of the data

In [ ]:
dataset["train"][0]
classes = [k.replace("_", " ") for k in dataset["train"].features["Label"].names]
print(type(dataset))
dataset = dataset.map(
    #add key text label to the dataset
    lambda x: {"text_label": f'{classes[x["Label"]]}'},
)
print(dataset["train"][0])